## Import Packages

In [117]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import numpy as np
import pandas as pd 

## Get List of State URLs

In [118]:
# Open chrome brower and go to state parent website
driver = webdriver.Chrome()
base_url = 'https://www.payscale.com/college-roi/state'
driver.get(base_url)

#Isolate body containing all state info
states = driver.find_elements_by_xpath('//*[@class="col-xs-12 col-sm-4"]/p')

state_urls = []
for state in states:
    #Isolate state name per state
    state_name = state.find_element_by_xpath( './/a' ).text
    
    #Generate URL from name and base_url
    state_urls.append([state_name, base_url + '/' + re.sub( r'[^\w\s]','',state_name.lower() ).replace( " ", "-" )])

## Prep Repository DF For State ROI Input

In [120]:
repo_df = pd.DataFrame({'state_name': [],\
                            'college_name':[],\
                           'college_type':[],\
                            'roi':[],\
                            'total_cost':[],\
                            'grad_rate':[],\
                            'yrs2grad':[],\
                            'financial_aid':[],\
                            'loan_amt':[],\
                            'perc_grant':[]})

## Scrape Each State

In [121]:
counter = 0
for url in state_urls: 
  
    #Unpack state-specific list containing name and url
    state_name, state_url = url
    print(state_name)
    
    #Go to state-specific url
    driver.get( state_url ) #enter state-specific url
    
    # Click review button to go to the review section
    oncampus_button = driver.find_element_by_xpath('//*[@id="roi-list-root"]/div/div/div/div/button')
    oncampus_button.click()

    roi_type_button = driver.find_element_by_xpath('//div[@id="roi-list-root"]/div/div/div[3]//button[1]')
    roi_type_button.click()

    load_more_button = driver.find_element_by_xpath('//*[@class="col-md-4 offset-md-2"]')
    load_more_button.click()
    
    #Must run twice to account for financial aid
    for aid in [True, False]:

        #Want to collect data with and without financial aid
        if aid:
            #Store data WITH financial aid
            financial_aid_button = driver.find_element_by_xpath('//*[@id="roi-list-root"]/div/div/div[2]//button[2]')
            financial_aid_button.click() 
        else:
            #Store data WITHOUT financial aid
            financial_aid_button = driver.find_element_by_xpath('//*[@id="roi-list-root"]/div/div/div[2]//button[1]')
            financial_aid_button.click() 

        #Wait for entire ROI to load
        wait_colleges = WebDriverWait( driver, 10 )
        colleges = wait_colleges.until( EC.presence_of_all_elements_located((By.XPATH, \
                                    '//*[@class="table college-roi-table table-bordered table-striped table-condensed"]/tbody/tr')) )

        #Scrape info per row (ie. college)
        for college in colleges:

            #Create dict for storage/export of data per college
            college_dict = {}

            #Scrape ROI first to determine if it is worth scraping rest of college info
            if aid:
                roi = re.sub( r'[$,%]','', college.find_element_by_xpath('./td[9]/span').text )
            else:
                roi = re.sub( r'[$,%]','', college.find_element_by_xpath('./td[7]/span').text )

            #Define additional xpaths and scrape college information
            text = college.find_element_by_xpath('./td[2]//div').text #roughly extract text
            college_name =  college.find_element_by_xpath('./td[2]//div').text #join college name
            college_type = re.sub( r'[)$,%(-]','', college_name.lower().split(r'(')[-1]) #clean up college type
            total_cost = re.sub( r'[)$,%(-]','', college.find_element_by_xpath('./td[11]/span').text )
            grad_rate = re.sub( r'[)$,%(-]','', college.find_element_by_xpath('./td[13]/span').text )
            yrs2grad = re.sub( r'[)$,%(-]','', college.find_element_by_xpath('./td[14]/span').text )

            # Scrap loan information if relevant
            if aid:
                loan_amt = re.sub( r'[$,%]','', college.find_element_by_xpath('./td[15]/span').text )
                perc_grant = re.sub( r'[$,%]','', college.find_element_by_xpath('./td[16]/span').text )
            else:
                loan_amt = 'N/A'
                perc_grant = 'N/A'    

            #Store all info to nested list to then create a df later
            college_df = pd.DataFrame({'state_name': [state_name],\
                            'college_name':[college_name],\
                           'college_type':[college_type],\
                            'roi':[roi],\
                            'total_cost':[total_cost],\
                            'grad_rate':[grad_rate],\
                            'yrs2grad':[yrs2grad],\
                            'financial_aid':[aid],\
                            'loan_amt':[loan_amt],\
                            'perc_grant':[perc_grant]})
            
            #Add college_df to repository df
            repo_df = pd.concat([repo_df, college_df], axis=0)

            #Keep track of number of colleges
            counter += 1
            
print(state_name)
print('='*25,counter, '='*25)

driver.close()

ALABAMA
ALABAMA
========================= 78 =========================
ALASKA
ALASKA
========================= 88 =========================
ARIZONA
ARIZONA
========================= 104 =========================
ARKANSAS
ARKANSAS
========================= 158 =========================
CALIFORNIA
CALIFORNIA
========================= 388 =========================
COLORADO
COLORADO
========================= 450 =========================
CONNECTICUT
CONNECTICUT
========================= 498 =========================
DELAWARE
DELAWARE
========================= 512 =========================
FLORIDA
FLORIDA
========================= 640 =========================
GEORGIA
GEORGIA
========================= 754 =========================
HAWAII
HAWAII
========================= 772 =========================
IDAHO
IDAHO
========================= 794 =========================
ILLINOIS
ILLINOIS
========================= 926 =========================
INDIANA
INDIANA
========================= 1040 =====

## Export Repository DF As CSV

In [122]:
repo_df.to_csv('college_roi_bystate.csv', index=None, header=True)